In [ ]:
class FinancialBasicInfo:
    """
    holds all fixed, starting parameters for the financial projection model.
    """
    def __init__(self,
                 current_age: int,
                 current_savings: float,
                 retirement_age: int,
                 death_age: int,
                 inflation_rate: float,
                 inflation_err_margin: float,
                 inv_growth_rate_avg: float,
                 inv_growth_err_margin: float):
        
        #core Inputs
        self.current_age = current_age
        self.current_savings = current_savings
        self.retirement_age = retirement_age
        self.death_age = death_age
        
        #inflation Rates
        self.inflation_rate = inflation_rate
        self.inflation_err_margin = inflation_err_margin
        
        #investment Growth Rates (average/baseline)
        self.inv_growth_rate_avg = inv_growth_rate_avg
        self.inv_growth_err_margin = inv_growth_err_margin
        
        #calculate derived properties
        self.calculate_derived_rates()

    def calculate_derived_rates(self):
        """calculates min/max rates for scenario analysis."""
        
        #investment growth scenarios
        self.inv_growth_rate_min = (
            self.inv_growth_rate_avg - self.inv_growth_err_margin
        )
        self.inv_growth_rate_max = (
            self.inv_growth_rate_avg + self.inv_growth_err_margin
        )
        
        #inflation scenarios
        self.inflation_rate_min = (
            self.inflation_rate - self.inflation_err_margin
        )
        self.inflation_rate_max = (
            self.inflation_rate + self.inflation_err_margin
        )

# --- Example Use ---

# 1. Instantiate the object with your initial data (using decimals for rates)
basic_info = FinancialBasicInfo(
    current_age=22,
    current_savings=0.0,
    retirement_age=65, # A common assumption, you can change this
    death_age=85,
    inflation_rate=0.03,        # 3%
    inflation_err_margin=0.02,  # 2%
    inv_growth_rate_avg=0.06,   # 6%
    inv_growth_err_margin=0.02, # 2%
)

# 2. Access the parameters and calculated values
print(f"Current Age: {basic_info.current_age}")
print(f"Baseline Growth Rate: {basic_info.inv_growth_rate_avg:.2%}")
print(f"Pessimistic Growth Rate (Min): {basic_info.inv_growth_rate_min:.2%}")
print(f"Optimistic Growth Rate (Max): {basic_info.inv_growth_rate_max:.2%}")

Current Age: 22
Baseline Growth Rate: 6.00%
Pessimistic Growth Rate (Min): 4.00%
Optimistic Growth Rate (Max): 8.00%


In [2]:
import pandas as pd
from typing import List, Dict, Any

# --- 1. CORE PARAMETER CLASS (From previous exchange, slightly enhanced) ---

class FinancialBasicInfo:
    """
    Holds all fixed, starting parameters for the financial projection model.
    """
    def __init__(self,
                 current_age: int,
                 current_savings: float,
                 retirement_age: int,
                 death_age: int,
                 inflation_rate: float,
                 inflation_err_margin: float,
                 inv_growth_rate_avg: float,
                 inv_growth_err_margin: float):
        
        # Core Inputs
        self.current_age = current_age
        self.current_savings = current_savings
        self.retirement_age = retirement_age
        self.death_age = death_age
        
        # Inflation Rates
        self.inflation_rate = inflation_rate
        self.inflation_err_margin = inflation_err_margin
        
        # Investment Growth Rates (Average/Baseline)
        self.inv_growth_rate_avg = inv_growth_rate_avg
        self.inv_growth_err_margin = inv_growth_err_margin
        
        # Calculate derived properties immediately
        self.calculate_derived_rates()

    def calculate_derived_rates(self):
        """Calculates min/max rates for scenario analysis."""
        
        # Investment Scenarios
        self.inv_growth_rate_min = (
            self.inv_growth_rate_avg - self.inv_growth_err_margin
        )
        self.inv_growth_rate_max = (
            self.inv_growth_rate_avg + self.inv_growth_err_margin
        )
        
        # Inflation Scenarios
        self.inflation_rate_min = (
            self.inflation_rate - self.inflation_err_margin
        )
        self.inflation_rate_max = (
            self.inflation_rate + self.inflation_err_margin
        )


# --- 2. PROJECTION INPUT CLASS ---

class ProjectionInfo:
    """
    Holds the starting inputs for a specific financial time-series (e.g., Savings or Spending).
    """
    def __init__(self, 
                 initial_amount: float, 
                 yoy_growth_rate: float, 
                 start_age: int, 
                 end_age: int,
                 name: str):
        
        self.initial_amount = initial_amount
        self.yoy_growth_rate = yoy_growth_rate
        self.start_age = start_age
        self.end_age = end_age
        self.name = name

    def __str__(self):
        return (f"{self.name} Projection from Age {self.start_age} to {self.end_age} "
                f"starting at ${self.initial_amount:,.2f} with {self.yoy_growth_rate:.2%} annual growth.")


# --- 3. PROJECTION CALCULATION ENGINE CLASS ---

class ProjectionEngine:
    """
    Handles the calculation of time-series progressions based on inputs.
    """
    def __init__(self, basic_info: FinancialBasicInfo):
        self.info = basic_info
        self.start_year = pd.Timestamp.now().year - self.info.current_age
    
    def calculate_progression(self, prog_info: ProjectionInfo) -> List[Dict[str, Any]]:
        """
        Calculates a yearly progression (Savings or Spending) and returns it as a list of dictionaries.
        """
        progression_data = []
        
        current_amount = prog_info.initial_amount
        
        # Iterate from the defined start age up to and including the end age
        for age in range(prog_info.start_age, prog_info.end_age + 1):
            
            # Calculate the corresponding year
            year = self.start_year + age
            
            progression_data.append({
                'Age': age,
                'Year': year,
                f'{prog_info.name} (Gross)': current_amount,
                f'{prog_info.name} Growth Rate': prog_info.yoy_growth_rate
            })
            
            # Calculate the amount for the NEXT year
            current_amount *= (1 + prog_info.yoy_growth_rate)
            
        return progression_data

    def adjust_for_inflation(self, 
                             initial_real_spending: float, 
                             target_start_age: int, 
                             inflation_rate: float) -> float:
        """
        Adjusts a current (real) spending amount to a future (nominal) amount 
        using inflation.
        
        Formula: Nominal Amount = Real Amount * (1 + Inflation Rate)^(Years to Retirement)
        """
        years_to_retirement = target_start_age - self.info.current_age
        
        if years_to_retirement < 0:
            return initial_real_spending # Already in retirement
        
        nominal_amount = initial_real_spending * (
            (1 + inflation_rate) ** years_to_retirement
        )
        return nominal_amount


# --- 4. EXECUTION AND DEMONSTRATION ---

# 1. Initialize Basic Info (using the same values as before)
info = FinancialBasicInfo(
    current_age=25, # Adjusted for a clearer example
    current_savings=5000.0,
    retirement_age=65,
    death_age=95,
    inflation_rate=0.03,
    inflation_err_margin=0.01, # Using 1% for clarity
    inv_growth_rate_avg=0.06,
    inv_growth_err_margin=0.02,
)

# 2. Initialize the Projection Engine
engine = ProjectionEngine(info)

# --- A. Savings Progression Setup ---
# Goal: Save $10,000 this year, growing contributions by 3% annually until retirement.
savings_info = ProjectionInfo(
    initial_amount=10000.0,
    yoy_growth_rate=0.03,
    start_age=info.current_age,
    end_age=info.retirement_age - 1, # Stop the year before retirement
    name='Savings Contribution'
)

# 3. Calculate Savings Progression
savings_progression = engine.calculate_progression(savings_info)
df_savings = pd.DataFrame(savings_progression)
print("\n--- SAVINGS PROGRESSION (Up to Retirement) ---")
print(df_savings.head())
print("...")
print(df_savings.tail())


# --- B. Retirement Spending Setup ---
# Goal: Desired annual spending of $50,000 IN TODAY'S DOLLARS (Real Spending)

# 1. Adjust the real spending goal for inflation to get the nominal starting amount
real_spending_goal = 50000.0
nominal_starting_spending = engine.adjust_for_inflation(
    initial_real_spending=real_spending_goal,
    target_start_age=info.retirement_age,
    inflation_rate=info.inflation_rate # Use the baseline inflation rate
)

# 2. Setup the Retirement Spending Progression
# Spending starts at retirement age and grows by inflation each year until death.
spending_info = ProjectionInfo(
    initial_amount=nominal_starting_spending,
    yoy_growth_rate=info.inflation_rate, # Spending grows by inflation
    start_age=info.retirement_age,
    end_age=info.death_age,
    name='Retirement Spending'
)

# 3. Calculate Spending Progression
spending_progression = engine.calculate_progression(spending_info)
df_spending = pd.DataFrame(spending_progression)

print(f"\n--- RETIREMENT SPENDING PROJECTION ---")
print(f"Goal: ${real_spending_goal:,.2f} (Today's Dollars)")
print(f"Calculated Nominal Start (Age {info.retirement_age}): ${nominal_starting_spending:,.2f}")
print("---")
print(df_spending.head())
print("...")
print(df_spending.tail())

# --- C. Final Combined Data Structure (Optional but Recommended) ---
# Merge the two DataFrames into one master projection table based on Age/Year
df_combined = pd.merge(
    df_savings, 
    df_spending, 
    on=['Age', 'Year'], 
    how='outer' # Use outer join to keep all ages from both tables
).sort_values(by='Age').fillna(0) # Fill NaNs (where savings ends or spending hasn't started)

print("\n--- COMBINED PROJECTION (Head & Tail) ---")
print(df_combined.head(3))
print("...")
print(df_combined.tail(3))



--- SAVINGS PROGRESSION (Up to Retirement) ---
   Age  Year  Savings Contribution (Gross)  Savings Contribution Growth Rate
0   25  2025                    10000.0000                              0.03
1   26  2026                    10300.0000                              0.03
2   27  2027                    10609.0000                              0.03
3   28  2028                    10927.2700                              0.03
4   29  2029                    11255.0881                              0.03
...
    Age  Year  Savings Contribution (Gross)  Savings Contribution Growth Rate
35   60  2060                  28138.624544                              0.03
36   61  2061                  28982.783280                              0.03
37   62  2062                  29852.266778                              0.03
38   63  2063                  30747.834782                              0.03
39   64  2064                  31670.269825                              0.03

--- RETIREMENT SP

In [4]:
import pandas as pd
from typing import List, Dict, Any

# --- 1. CORE PARAMETER CLASS ---

class FinancialBasicInfo:
    """
    Holds all fixed, starting parameters for the financial projection model.
    """
    def __init__(self,
                 current_age: int,
                 current_savings: float,
                 retirement_age: int,
                 death_age: int,
                 inflation_rate: float,
                 inflation_err_margin: float,
                 inv_growth_rate_avg: float,
                 inv_growth_err_margin: float):
        
        # Core Inputs
        self.current_age = current_age
        self.current_savings = current_savings
        self.retirement_age = retirement_age
        self.death_age = death_age
        
        # Inflation Rates
        self.inflation_rate = inflation_rate
        self.inflation_err_margin = inflation_err_margin
        
        # Investment Growth Rates (Average/Baseline)
        self.inv_growth_rate_avg = inv_growth_rate_avg
        self.inv_growth_err_margin = inv_growth_err_margin
        
        # Calculate derived properties immediately
        self.calculate_derived_rates()

    def calculate_derived_rates(self):
        """Calculates min/max rates for scenario analysis."""
        
        # Investment Scenarios
        self.inv_growth_rate_min = (
            self.inv_growth_rate_avg - self.inv_growth_err_margin
        )
        self.inv_growth_rate_max = (
            self.inv_growth_rate_avg + self.inv_growth_err_margin
        )
        
        # Inflation Scenarios
        self.inflation_rate_min = (
            self.inflation_rate - self.inflation_err_margin
        )
        self.inflation_rate_max = (
            self.inflation_rate + self.inflation_err_margin
        )


# --- 2. PROJECTION INPUT CLASS ---

class ProjectionInfo:
    """
    Holds the starting inputs for a specific financial time-series (e.g., Savings or Spending).
    """
    def __init__(self, 
                 initial_amount: float, 
                 yoy_growth_rate: float, 
                 start_age: int, 
                 end_age: int,
                 name: str):
        
        self.initial_amount = initial_amount
        self.yoy_growth_rate = yoy_growth_rate
        self.start_age = start_age
        self.end_age = end_age
        self.name = name

    def __str__(self):
        return (f"{self.name} Projection from Age {self.start_age} to {self.end_age} "
                f"starting at ${self.initial_amount:,.2f} with {self.yoy_growth_rate:.2%} annual growth.")


# --- 3. PROJECTION CALCULATION ENGINE CLASS ---

class ProjectionEngine:
    """
    Handles the calculation of time-series progressions based on inputs.
    """
    def __init__(self, basic_info: FinancialBasicInfo):
        self.info = basic_info
        # Calculate the year of the current age, used to project future years
        self.start_year = pd.Timestamp.now().year - self.info.current_age
    
    def get_scenarios(self) -> Dict[str, Dict[str, float]]:
        """
        Defines and returns the three main scenario rate combinations.
        """
        return {
            'Average Case (Avg)': {
                'inv_growth': self.info.inv_growth_rate_avg,
                'inflation': self.info.inflation_rate
            },
            'Optimistic Case (Good)': {
                # High Growth, Low Inflation (Best outcome)
                'inv_growth': self.info.inv_growth_rate_max,
                'inflation': self.info.inflation_rate_min
            },
            'Pessimistic Case (Bad)': {
                # Low Growth, High Inflation (Worst outcome)
                'inv_growth': self.info.inv_growth_rate_min,
                'inflation': self.info.inflation_rate_max
            }
        }

    def calculate_progression(self, prog_info: ProjectionInfo) -> List[Dict[str, Any]]:
        """
        Calculates a yearly progression (Savings or Spending) and returns it as a list of dictionaries.
        """
        progression_data = []
        
        current_amount = prog_info.initial_amount
        
        # Iterate from the defined start age up to and including the end age
        for age in range(prog_info.start_age, prog_info.end_age + 1):
            
            # Calculate the corresponding year
            year = self.start_year + age
            
            progression_data.append({
                'Age': age,
                'Year': year,
                f'{prog_info.name} (Gross)': current_amount,
                f'{prog_info.name} Growth Rate': prog_info.yoy_growth_rate
            })
            
            # Calculate the amount for the NEXT year
            current_amount *= (1 + prog_info.yoy_growth_rate)
            
        return progression_data

    def adjust_for_inflation(self, 
                             initial_real_spending: float, 
                             target_start_age: int, 
                             inflation_rate: float) -> float:
        """
        Adjusts a current (real) spending amount to a future (nominal) amount 
        using inflation.
        """
        years_to_retirement = target_start_age - self.info.current_age
        
        if years_to_retirement <= 0:
            return initial_real_spending
        
        nominal_amount = initial_real_spending * (
            (1 + inflation_rate) ** years_to_retirement
        )
        return nominal_amount


# --- 4. EXECUTION AND DEMONSTRATION ---

# 1. Initialize Basic Info 
info = FinancialBasicInfo(
    current_age=25,
    current_savings=5000.0,
    retirement_age=65,
    death_age=95,
    inflation_rate=0.03,        # 3% Baseline Inflation
    inflation_err_margin=0.01,  # +/- 1% Inflation Error Margin
    inv_growth_rate_avg=0.06,   # 6% Baseline Growth
    inv_growth_err_margin=0.02, # +/- 2% Growth Error Margin
)

# 2. Initialize the Projection Engine
engine = ProjectionEngine(info)
scenarios = engine.get_scenarios()

# --- A. Savings Progression Setup (Independent of Market Scenarios) ---
savings_info = ProjectionInfo(
    initial_amount=10000.0,
    yoy_growth_rate=0.03,
    start_age=info.current_age,
    end_age=info.retirement_age - 1,
    name='Savings Contribution'
)
savings_progression = engine.calculate_progression(savings_info)
df_savings = pd.DataFrame(savings_progression)
# FIX: Do NOT set index or drop 'Year' here. Keep all columns for merging later.

print("\n--- SAVINGS CONTRIBUTION PROGRESSION (Single Baseline) ---")
print(df_savings[['Age', 'Year', 'Savings Contribution (Gross)']].head())


# --- B. Retirement Spending Setup (Scenario Dependent) ---
real_spending_goal = 50000.0

# FIX: Create the base DataFrame for combined spending using the columns we know exist.
df_spending_combined = df_savings[['Age', 'Year']].copy() 

print(f"\n--- RETIREMENT SPENDING PROJECTION (Goal: ${real_spending_goal:,.2f} Today's Dollars) ---")

for scenario_name, rates in scenarios.items():
    inflation_rate = rates['inflation']
    
    # 1. Calculate the Nominal starting spending for this scenario
    nominal_starting_spending = engine.adjust_for_inflation(
        initial_real_spending=real_spending_goal,
        target_start_age=info.retirement_age,
        inflation_rate=inflation_rate 
    )
    
    # 2. Setup the Retirement Spending Progression for this scenario
    spending_info = ProjectionInfo(
        initial_amount=nominal_starting_spending,
        yoy_growth_rate=inflation_rate, # Spending grows by the scenario's inflation rate
        start_age=info.retirement_age,
        end_age=info.death_age,
        name=f'Retirement Spending ({scenario_name})'
    )
    
    # 3. Calculate Progression and create a temporary DataFrame
    spending_progression = engine.calculate_progression(spending_info)
    df_temp = pd.DataFrame(spending_progression)
    df_temp = df_temp.rename(columns={
        f'Retirement Spending ({scenario_name}) (Gross)': f'Spending Nominal ({scenario_name})'
    })
    
    # Merge the spending progression into the combined DataFrame
    # Use 'Age' as the primary join key
    df_spending_combined = pd.merge(
        df_spending_combined, 
        df_temp[['Age', f'Spending Nominal ({scenario_name})']], 
        on='Age', 
        how='outer' # Keep all ages (savings ages and spending ages)
    )

    print(f"[{scenario_name} | Inf: {inflation_rate:.2%}] Nominal Start at Age {info.retirement_age}: ${nominal_starting_spending:,.2f}")

# --- C. Final Combined Data Structure ---
# Re-merge the single savings stream with the multi-scenario spending streams
# Since df_savings now contains 'Age' and 'Year' as columns, this merge works cleanly.
df_combined = pd.merge(
    df_savings.drop(columns='Retirement Spending (Average Case (Avg)) Growth Rate', errors='ignore'), # Drop the growth rate column from savings to prevent conflict
    df_spending_combined, 
    on=['Age', 'Year'], 
    how='outer'
).sort_values(by='Age').fillna(0)

# Clean up final dataframe (remove duplicate year column if created by outer merge)
df_combined = df_combined.loc[:, ~df_combined.columns.duplicated()].copy()


print("\n--- COMBINED PROJECTION (Retirement Spending Scenarios) ---")
print(df_combined[['Age', 'Year', 'Savings Contribution (Gross)', 'Spending Nominal (Average Case (Avg))', 'Spending Nominal (Optimistic Case (Good))', 'Spending Nominal (Pessimistic Case (Bad))']].head(3))
print("...")
print(df_combined[['Age', 'Year', 'Savings Contribution (Gross)', 'Spending Nominal (Average Case (Avg))', 'Spending Nominal (Optimistic Case (Good))', 'Spending Nominal (Pessimistic Case (Bad))']].tail(3))


--- SAVINGS CONTRIBUTION PROGRESSION (Single Baseline) ---
   Age  Year  Savings Contribution (Gross)
0   25  2025                    10000.0000
1   26  2026                    10300.0000
2   27  2027                    10609.0000
3   28  2028                    10927.2700
4   29  2029                    11255.0881

--- RETIREMENT SPENDING PROJECTION (Goal: $50,000.00 Today's Dollars) ---
[Average Case (Avg) | Inf: 3.00%] Nominal Start at Age 65: $163,101.89
[Optimistic Case (Good) | Inf: 2.00%] Nominal Start at Age 65: $110,401.98
[Pessimistic Case (Bad) | Inf: 4.00%] Nominal Start at Age 65: $240,051.03

--- COMBINED PROJECTION (Retirement Spending Scenarios) ---
   Age    Year  Savings Contribution (Gross)  \
0   25  2025.0                       10000.0   
1   26  2026.0                       10300.0   
2   27  2027.0                       10609.0   

   Spending Nominal (Average Case (Avg))  \
0                                    0.0   
1                                    0.0   


In [7]:
import pandas as pd
import plotly.graph_objects as go
from typing import Dict, List, Any
# Assuming the classes from financial_model.py (FinancialBasicInfo, ProjectionEngine, ProjectionInfo) 
# are available or pasted above this code block for execution.

# Note: For this script to run, the classes from financial_model.py must be defined in the environment.

# --- 1. CORE PARAMETER CLASS (RE-DEFINED FOR SELF-CONTAINMENT) ---

class FinancialBasicInfo:
    """Holds all fixed, starting parameters for the financial projection model."""
    def __init__(self, current_age: int, current_savings: float, retirement_age: int, death_age: int,
                 inflation_rate: float, inflation_err_margin: float, inv_growth_rate_avg: float,
                 inv_growth_err_margin: float):
        self.current_age = current_age
        self.current_savings = current_savings
        self.retirement_age = retirement_age
        self.death_age = death_age
        self.inflation_rate = inflation_rate
        self.inflation_err_margin = inflation_err_margin
        self.inv_growth_rate_avg = inv_growth_rate_avg
        self.inv_growth_err_margin = inv_growth_err_margin
        self.calculate_derived_rates()

    def calculate_derived_rates(self):
        """Calculates min/max rates for scenario analysis."""
        self.inv_growth_rate_min = self.inv_growth_rate_avg - self.inv_growth_err_margin
        self.inv_growth_rate_max = self.inv_growth_rate_avg + self.inv_growth_err_margin
        self.inflation_rate_min = self.inflation_rate - self.inflation_err_margin
        self.inflation_rate_max = self.inflation_rate + self.inflation_err_margin

class ProjectionInfo:
    """Holds the starting inputs for a specific financial time-series."""
    def __init__(self, initial_amount: float, yoy_growth_rate: float, start_age: int, end_age: int, name: str):
        self.initial_amount = initial_amount
        self.yoy_growth_rate = yoy_growth_rate
        self.start_age = start_age
        self.end_age = end_age
        self.name = name

class ProjectionEngine:
    """Handles the calculation of time-series progressions based on inputs."""
    def __init__(self, basic_info: FinancialBasicInfo):
        self.info = basic_info
        self.start_year = pd.Timestamp.now().year - self.info.current_age
    
    def get_scenarios(self) -> Dict[str, Dict[str, float]]:
        """Defines and returns the three main scenario rate combinations."""
        return {
            'Average Case (Avg)': {'inv_growth': self.info.inv_growth_rate_avg, 'inflation': self.info.inflation_rate},
            'Optimistic Case (Good)': {'inv_growth': self.info.inv_growth_rate_max, 'inflation': self.info.inflation_rate_min},
            'Pessimistic Case (Bad)': {'inv_growth': self.info.inv_growth_rate_min, 'inflation': self.info.inflation_rate_max}
        }

    def calculate_progression(self, prog_info: ProjectionInfo) -> List[Dict[str, Any]]:
        """Calculates a yearly progression (Savings or Spending)."""
        progression_data = []
        current_amount = prog_info.initial_amount
        
        for age in range(prog_info.start_age, prog_info.end_age + 1):
            year = self.start_year + age
            progression_data.append({
                'Age': age,
                'Year': year,
                f'{prog_info.name} (Gross)': current_amount,
                f'{prog_info.name} Growth Rate': prog_info.yoy_growth_rate
            })
            current_amount *= (1 + prog_info.yoy_growth_rate)
            
        return progression_data

    def adjust_for_inflation(self, initial_real_spending: float, target_start_age: int, inflation_rate: float) -> float:
        """Adjusts a current (real) spending amount to a future (nominal) amount using inflation."""
        years_to_retirement = target_start_age - self.info.current_age
        if years_to_retirement <= 0:
            return initial_real_spending
        nominal_amount = initial_real_spending * ((1 + inflation_rate) ** years_to_retirement)
        return nominal_amount

# ----------------------------------------------------------------------
# --- PLOTLY CHART FUNCTION ---
# ----------------------------------------------------------------------

def create_projection_chart(df_combined: pd.DataFrame, retirement_age: int) -> go.Figure:
    """
    Creates an interactive Plotly graph showing the savings contribution, 
    cumulative investment balance, and three retirement spending scenarios over time.
    """
    fig = go.Figure()
    
    # --- 1. Savings Contribution (Bar Chart) ---
    fig.add_trace(go.Bar(
        x=df_combined['Age'],
        y=df_combined['Savings Contribution (Gross)'],
        name='Annual Savings Contribution',
        marker_color='#5cb85c', # Green
        opacity=0.8,
        hovertemplate="Age %{x}: %{y:$,.2f}<extra></extra>"
    ))

    # --- 2. Cumulative Investment Balance Scenarios (New Lines) ---
    # Colors are chosen for conceptual clarity: Red for low outcome, Green for high outcome.
    balance_cols = [
        ('Balance Nominal (Pessimistic Case (Bad))', 'Pessimistic Balance (Low Growth)', '#800000'), # Dark Red (Worst financial outcome)
        ('Balance Nominal (Average Case (Avg))', 'Average Balance', '#007bff'),                      # Blue
        ('Balance Nominal (Optimistic Case (Good))', 'Optimistic Balance (High Growth)', '#28a745') # Bright Green (Best financial outcome)
    ]
    
    for col_name, trace_name, color in balance_cols:
        fig.add_trace(go.Scatter(
            x=df_combined['Age'],
            y=df_combined[col_name],
            mode='lines',
            name=trace_name,
            line=dict(color=color, width=4, dash='solid'),
            hovertemplate=f"{trace_name}: %{{y:$,.2f}}<extra>Age %{{x}}</extra>"
        ))

    # --- 3. Retirement Spending Scenarios (Updated Colors/Order) ---
    # The high line (Pessimistic Spending/High Inflation) correctly uses the warning color.
    spending_cols = [
        ('Spending Nominal (Pessimistic Case (Bad))', 'Pessimistic Spending (High Inflation)', 'red'), # Highest required spending (Worst Case Requirement)
        ('Spending Nominal (Average Case (Avg))', 'Average Spending (Baseline Inflation)', 'orange'),
        ('Spending Nominal (Optimistic Case (Good))', 'Optimistic Spending (Low Inflation)', 'blue')  # Lowest required spending (Best Case Requirement)
    ]

    for col_name, trace_name, color in spending_cols:
        # Filter for ages at and after retirement for spending lines
        spending_data = df_combined[df_combined['Age'] >= retirement_age]
        
        fig.add_trace(go.Scatter(
            x=spending_data['Age'],
            y=spending_data[col_name],
            mode='lines',
            name=trace_name,
            line=dict(color=color, width=3, dash='dash'),
            hovertemplate=f"{trace_name}: %{{y:$,.2f}}<extra>Age %{{x}}</extra>"
        ))

    # --- 4. Layout and Annotations ---
    fig.update_layout(
        title='Financial Progression: Balances, Contributions, and Spending Scenarios',
        xaxis_title='Age (Years)',
        yaxis_title='Nominal Dollar Amount (USD)',
        hovermode='x unified',
        template='plotly_white',
        barmode='overlay',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Highlight Retirement Age
    fig.add_vline(x=retirement_age, line_width=2, line_dash="dash", line_color="gray",
                  annotation_text=f"Retirement ({retirement_age})", 
                  annotation_position="top left")
    
    # Format Y-axis to display currency
    fig.update_yaxes(tickprefix='$', tickformat=',.0f')
    
    return fig

# ----------------------------------------------------------------------
# --- EXECUTION: GENERATE DATA AND PLOT ---
# ----------------------------------------------------------------------

# 1. Initialize Basic Info 
info = FinancialBasicInfo(
    current_age=25,
    current_savings=5000.0,
    retirement_age=65,
    death_age=95,
    inflation_rate=0.03,        # 3% Baseline Inflation
    inflation_err_margin=0.01,  # +/- 1% Inflation Error Margin
    inv_growth_rate_avg=0.06,   # 6% Baseline Growth
    inv_growth_err_margin=0.02, # +/- 2% Growth Error Margin
)

# 2. Initialize the Projection Engine
engine = ProjectionEngine(info)
scenarios: Dict[str, Dict[str, float]] = engine.get_scenarios()

# --- A. Savings Progression Setup (for annual contribution) ---
savings_info = ProjectionInfo(
    initial_amount=10000.0,
    yoy_growth_rate=0.03,
    start_age=info.current_age,
    end_age=info.retirement_age - 1,
    name='Savings Contribution'
)
savings_progression = engine.calculate_progression(savings_info)
df_savings = pd.DataFrame(savings_progression)


# --- B. Retirement Spending Setup (Scenario Dependent) ---
real_spending_goal = 50000.0
df_spending_combined = df_savings[['Age', 'Year']].copy() 

for scenario_name, rates in scenarios.items():
    inflation_rate = rates['inflation']
    
    nominal_starting_spending = engine.adjust_for_inflation(
        initial_real_spending=real_spending_goal,
        target_start_age=info.retirement_age,
        inflation_rate=inflation_rate 
    )
    
    spending_info = ProjectionInfo(
        initial_amount=nominal_starting_spending,
        yoy_growth_rate=inflation_rate,
        start_age=info.retirement_age,
        end_age=info.death_age,
        name=f'Retirement Spending ({scenario_name})'
    )
    
    spending_progression = engine.calculate_progression(spending_info)
    df_temp = pd.DataFrame(spending_progression)
    df_temp = df_temp.rename(columns={
        f'Retirement Spending ({scenario_name}) (Gross)': f'Spending Nominal ({scenario_name})'
    })
    
    df_spending_combined = pd.merge(
        df_spending_combined, 
        df_temp[['Age', f'Spending Nominal ({scenario_name})']], 
        on='Age', 
        how='outer'
    )

# --- C. Final Combined Data Structure (Merge Contributions and Spending) ---
df_combined = pd.merge(
    df_savings.drop(columns=[col for col in df_savings.columns if 'Growth Rate' in col], errors='ignore'),
    df_spending_combined, 
    on=['Age', 'Year'], 
    how='outer'
).sort_values(by='Age').fillna(0)
df_combined = df_combined.loc[:, ~df_combined.columns.duplicated()].copy()


# --- D. Cumulative Balance Calculation (Investment Scenarios) ---
# Get annual savings contributions aligned by age
df_contributions = df_combined[['Age', 'Savings Contribution (Gross)']].set_index('Age').fillna(0)

for scenario_name, rates in scenarios.items():
    inv_growth_rate = rates['inv_growth']
    balance_column = f'Balance Nominal ({scenario_name})'
    
    current_balance = info.current_savings # Start with initial savings
    
    # Create a list to hold the yearly balances for this scenario
    yearly_balances = []
    
    # Iterate through all ages in the projection period
    for age in df_combined['Age']:
        # Get contribution for the year (will be 0 after retirement)
        # Note: We assume the last contribution is made at retirement_age - 1
        contribution = df_contributions.loc[age, 'Savings Contribution (Gross)'] if age < info.retirement_age else 0
        
        # Calculate balance for this year: (Balance + Contribution) * (1 + Growth Rate)
        current_balance = (current_balance + contribution) * (1 + inv_growth_rate)
        
        yearly_balances.append({
            'Age': age,
            balance_column: current_balance
        })

    # Convert to DataFrame and merge
    df_balance_temp = pd.DataFrame(yearly_balances)
    df_combined = pd.merge(df_combined, df_balance_temp, on='Age', how='left').fillna(0)

# Re-sort for safety
df_combined = df_combined.sort_values(by='Age')

# --- E. Generate Plotly Chart ---
fig = create_projection_chart(df_combined, info.retirement_age)

# To display the interactive graph, you typically save it as an HTML file.
# You can view the output in a web browser.
output_file = "financial_projection_chart.html"
fig.write_html(output_file, full_html=True)

print(f"\n--- Plotly Chart Generated ---")
print(f"Interactive chart saved to: {output_file}")
print("Open this file in a browser to view the graph.")



--- Plotly Chart Generated ---
Interactive chart saved to: financial_projection_chart.html
Open this file in a browser to view the graph.


In [8]:
fig.show()

In [9]:
import pandas as pd
import plotly.graph_objects as go
from typing import Dict, List, Any

# Note: The classes FinancialBasicInfo, ProjectionInfo, and ProjectionEngine are 
# included here for self-containment, but ideally would be imported from a separate module.

# --- 1. CORE PARAMETER CLASS (RE-DEFINED FOR SELF-CONTAINMENT) ---

class FinancialBasicInfo:
    """Holds all fixed, starting parameters for the financial projection model."""
    def __init__(self, current_age: int, current_savings: float, retirement_age: int, death_age: int,
                 inflation_rate: float, inflation_err_margin: float, inv_growth_rate_avg: float,
                 inv_growth_err_margin: float):
        self.current_age = current_age
        self.current_savings = current_savings
        self.retirement_age = retirement_age
        self.death_age = death_age
        self.inflation_rate = inflation_rate
        self.inflation_err_margin = inflation_err_margin
        self.inv_growth_rate_avg = inv_growth_rate_avg
        self.inv_growth_err_margin = inv_growth_err_margin
        self.calculate_derived_rates()

    def calculate_derived_rates(self):
        """Calculates min/max rates for scenario analysis."""
        self.inv_growth_rate_min = self.inv_growth_rate_avg - self.inv_growth_err_margin
        self.inv_growth_rate_max = self.inv_growth_rate_avg + self.inv_growth_err_margin
        self.inflation_rate_min = self.inflation_rate - self.inflation_err_margin
        self.inflation_rate_max = self.inflation_rate + self.inflation_err_margin

class ProjectionInfo:
    """Holds the starting inputs for a specific financial time-series."""
    def __init__(self, initial_amount: float, yoy_growth_rate: float, start_age: int, end_age: int, name: str):
        self.initial_amount = initial_amount
        self.yoy_growth_rate = yoy_growth_rate
        self.start_age = start_age
        self.end_age = end_age
        self.name = name

class ProjectionEngine:
    """Handles the calculation of time-series progressions based on inputs."""
    def __init__(self, basic_info: FinancialBasicInfo):
        self.info = basic_info
        self.start_year = pd.Timestamp.now().year - self.info.current_age
    
    def get_scenarios(self) -> Dict[str, Dict[str, float]]:
        """Defines and returns the three main scenario rate combinations."""
        # Note: This is now just used to easily retrieve the rates in the execution block.
        return {
            'Average Case (Avg)': {'inv_growth': self.info.inv_growth_rate_avg, 'inflation': self.info.inflation_rate},
            'Optimistic Case (Good)': {'inv_growth': self.info.inv_growth_rate_max, 'inflation': self.info.inflation_rate_min},
            'Pessimistic Case (Bad)': {'inv_growth': self.info.inv_growth_rate_min, 'inflation': self.info.inflation_rate_max}
        }

    def calculate_progression(self, prog_info: ProjectionInfo) -> List[Dict[str, Any]]:
        """Calculates a yearly progression (Savings or Spending). Used here only for Contributions."""
        progression_data = []
        current_amount = prog_info.initial_amount
        
        for age in range(prog_info.start_age, prog_info.end_age + 1):
            year = self.start_year + age
            progression_data.append({
                'Age': age,
                'Year': year,
                f'{prog_info.name} (Gross)': current_amount,
                f'{prog_info.name} Growth Rate': prog_info.yoy_growth_rate
            })
            current_amount *= (1 + prog_info.yoy_growth_rate)
            
        return progression_data

    def adjust_for_inflation(self, initial_real_spending: float, target_start_age: int, inflation_rate: float) -> float:
        """Adjusts a current (real) spending amount to a future (nominal) amount using inflation."""
        years_to_retirement = target_start_age - self.info.current_age
        if years_to_retirement <= 0:
            return initial_real_spending
        nominal_amount = initial_real_spending * ((1 + inflation_rate) ** years_to_retirement)
        return nominal_amount

# ----------------------------------------------------------------------
# --- PLOTLY CHART FUNCTION ---
# ----------------------------------------------------------------------

def create_projection_chart(df_combined: pd.DataFrame, retirement_age: int, death_age: int) -> go.Figure:
    """
    Creates an interactive Plotly graph showing the net projected retirement balance 
    for the three combined scenarios (Accumulation & Decumulation).
    """
    fig = go.Figure()
    
    # Define the three net scenarios for plotting
    net_cols = [
        ('Balance (Worst Case Net)', 'Worst Case Net (Min Savings - Max Spending)', 'red'), 
        ('Balance (Average Case Net)', 'Average Case Net (Avg Savings - Avg Spending)', 'blue'),
        ('Balance (Best Case Net)', 'Best Case Net (Max Savings - Min Spending)', 'green')
    ]
    
    # --- 1. Net Balance Scenarios (Lines) ---
    for col_name, trace_name, color in net_cols:
        fig.add_trace(go.Scatter(
            x=df_combined['Age'],
            y=df_combined[col_name],
            mode='lines',
            name=trace_name,
            line=dict(color=color, width=4, dash='solid'),
            hovertemplate=f"{trace_name}: %{{y:$,.2f}}<extra>Age %{{x}}</extra>"
        ))

    # --- 2. Layout and Annotations ---
    fig.update_layout(
        title='Projected Retirement Balance: Combined Investment & Spending Scenarios',
        xaxis_title='Age (Years)',
        yaxis_title='Projected End-of-Year Balance (Nominal USD)',
        hovermode='x unified',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Highlight Retirement Age
    fig.add_vline(x=retirement_age, line_width=2, line_dash="dash", line_color="gray",
                  annotation_text=f"Retirement ({retirement_age})", 
                  annotation_position="top left")

    # Add a zero line to easily see when the balance is depleted
    fig.add_hline(y=0, line_width=2, line_color="black", line_dash="dot")
    
    # Format Y-axis to display currency
    fig.update_yaxes(tickprefix='$', tickformat=',.0f')
    
    return fig

# ----------------------------------------------------------------------
# --- EXECUTION: GENERATE DATA AND PLOT ---
# ----------------------------------------------------------------------

# 1. Initialize Basic Info (Using previous setup values)
info = FinancialBasicInfo(
    current_age=25,
    current_savings=5000.0,
    retirement_age=65,
    death_age=95,
    inflation_rate=0.03,        # 3% Baseline Inflation
    inflation_err_margin=0.01,  # +/- 1% Inflation Error Margin
    inv_growth_rate_avg=0.06,   # 6% Baseline Growth
    inv_growth_err_margin=0.02, # +/- 2% Growth Error Margin
)

# 2. Initialize the Projection Engine
engine = ProjectionEngine(info)

# --- A. Savings Progression Setup (for annual contribution) ---
savings_info = ProjectionInfo(
    initial_amount=10000.0,
    yoy_growth_rate=0.03, # Annual growth rate of contributions
    start_age=info.current_age,
    end_age=info.retirement_age - 1,
    name='Savings Contribution'
)
savings_progression = engine.calculate_progression(savings_info)
df_savings = pd.DataFrame(savings_progression)

# Get annual savings contributions aligned by age
df_contributions = df_savings[['Age', 'Savings Contribution (Gross)']].set_index('Age').fillna(0)

# --- B. Define the Three Combined Net Scenarios ---
combined_scenarios = {
    'Worst Case Net': {
        'inv_growth': info.inv_growth_rate_min,   # Min Savings Growth
        'inflation': info.inflation_rate_max      # Max Spending Inflation
    },
    'Average Case Net': {
        'inv_growth': info.inv_growth_rate_avg,  # Avg Savings Growth
        'inflation': info.inflation_rate         # Avg Spending Inflation
    },
    'Best Case Net': {
        'inv_growth': info.inv_growth_rate_max,   # Max Savings Growth
        'inflation': info.inflation_rate_min      # Min Spending Inflation
    }
}

real_spending_goal = 50000.0
all_ages = range(info.current_age, info.death_age + 1)
df_combined = pd.DataFrame({'Age': all_ages})

# --- C. Full Accumulation & Decumulation Simulation ---

for scenario_name, rates in combined_scenarios.items():
    inv_growth_rate = rates['inv_growth']
    inflation_rate = rates['inflation']
    
    current_balance = info.current_savings # Start with initial savings
    
    # Calculate the nominal spending required at retirement (Age 65) for this inflation scenario
    nominal_spending_at_retirement = engine.adjust_for_inflation(
        initial_real_spending=real_spending_goal,
        target_start_age=info.retirement_age,
        inflation_rate=inflation_rate 
    )
    
    current_annual_spending = 0.0 # Will be set to the nominal starting spending at retirement age
    yearly_data = []

    for age in all_ages:
        
        # --- 1. DETERMINE CASH FLOW ---
        
        # Pre-Retirement Phase (Accumulation)
        if age < info.retirement_age:
            contribution = df_contributions.loc[age, 'Savings Contribution (Gross)']
            required_spending = 0.0
            cash_flow = contribution # Positive cash flow
            
        # Post-Retirement Phase (Decumulation)
        elif age >= info.retirement_age:
            
            # Calculate the nominal required spending for this year
            if age == info.retirement_age:
                 required_spending = nominal_spending_at_retirement
            else:
                 # Spending grows by the scenario's inflation rate each subsequent year
                 required_spending = current_annual_spending * (1 + inflation_rate)
            
            # Update the tracked annual spending for the next iteration
            current_annual_spending = required_spending
            
            cash_flow = -required_spending # Negative cash flow (withdrawal)
        
        # --- 2. UPDATE BALANCE ---
        
        # If the balance is already 0, it remains 0 (cannot go more negative than 0 in this model)
        if current_balance <= 0:
            final_balance = 0
        else:
            # Step A: Apply cash flow (contribution or withdrawal)
            balance_after_cash_flow = current_balance + cash_flow
            
            # Step B: Apply investment growth for the year
            # Note: We still apply growth even if the balance is negative, but the final_balance is capped at 0 for plotting
            current_balance = balance_after_cash_flow * (1 + inv_growth_rate)
            final_balance = max(0, current_balance)

        yearly_data.append({
            'Age': age,
            f'Balance ({scenario_name})': final_balance
        })

    # Merge the results back to the combined DataFrame
    df_temp = pd.DataFrame(yearly_data)
    df_combined = pd.merge(df_combined, df_temp[['Age', f'Balance ({scenario_name})']], on='Age', how='left')

df_combined = df_combined.sort_values(by='Age')

# --- D. Generate Plotly Chart ---
fig = create_projection_chart(df_combined, info.retirement_age, info.death_age)

# To display the interactive graph, you typically save it as an HTML file.
# You can view the output in a web browser.
output_file = "financial_projection_chart.html"
fig.write_html(output_file, full_html=True)

print(f"\n--- Plotly Chart Generated ---")
print(f"Interactive chart saved to: {output_file}")
print("Open this file in a browser to view the graph.")



--- Plotly Chart Generated ---
Interactive chart saved to: financial_projection_chart.html
Open this file in a browser to view the graph.


## editing this to try to match my excel

In [13]:

# ----------------------------------------------------------------------
# --- EXECUTION: GENERATE DATA AND PLOT ---
# ----------------------------------------------------------------------

# 1. Initialize Basic Info (Using previous setup values)
info = FinancialBasicInfo(
    current_age=22,
    current_savings=0.01, #need to fix this error, 0 isn't allowed
    retirement_age=65,
    death_age=95,
    inflation_rate=0.03,        # 3% Baseline Inflation
    inflation_err_margin=0.01,  # +/- 1% Inflation Error Margin
    inv_growth_rate_avg=0.06,   # 6% Baseline Growth
    inv_growth_err_margin=0.02, # +/- 2% Growth Error Margin
)

# 2. Initialize the Projection Engine
engine = ProjectionEngine(info)

# --- A. Savings Progression Setup (for annual contribution) ---
savings_info = ProjectionInfo(
    initial_amount=5000.0,
    yoy_growth_rate=0.03, # Annual growth rate of contributions
    start_age=info.current_age,
    end_age=info.retirement_age - 1,
    name='Savings Contribution'
)
savings_progression = engine.calculate_progression(savings_info)
df_savings = pd.DataFrame(savings_progression)

# Get annual savings contributions aligned by age
df_contributions = df_savings[['Age', 'Savings Contribution (Gross)']].set_index('Age').fillna(0)

# --- B. Define the Three Combined Net Scenarios ---
combined_scenarios = {
    'Worst Case Net': {
        'inv_growth': info.inv_growth_rate_min,   # Min Savings Growth
        'inflation': info.inflation_rate_max      # Max Spending Inflation
    },
    'Average Case Net': {
        'inv_growth': info.inv_growth_rate_avg,  # Avg Savings Growth
        'inflation': info.inflation_rate         # Avg Spending Inflation
    },
    'Best Case Net': {
        'inv_growth': info.inv_growth_rate_max,   # Max Savings Growth
        'inflation': info.inflation_rate_min      # Min Spending Inflation
    }
}

real_spending_goal = 50000.0
all_ages = range(info.current_age, info.death_age + 1)
df_combined = pd.DataFrame({'Age': all_ages})

# --- C. Full Accumulation & Decumulation Simulation ---

for scenario_name, rates in combined_scenarios.items():
    inv_growth_rate = rates['inv_growth']
    inflation_rate = rates['inflation']
    
    current_balance = info.current_savings # Start with initial savings
    
    # Calculate the nominal spending required at retirement (Age 65) for this inflation scenario
    nominal_spending_at_retirement = engine.adjust_for_inflation(
        initial_real_spending=real_spending_goal,
        target_start_age=info.retirement_age,
        inflation_rate=inflation_rate 
    )
    
    current_annual_spending = 0.0 # Will be set to the nominal starting spending at retirement age
    yearly_data = []

    for age in all_ages:
        
        # --- 1. DETERMINE CASH FLOW ---
        
        # Pre-Retirement Phase (Accumulation)
        if age < info.retirement_age:
            contribution = df_contributions.loc[age, 'Savings Contribution (Gross)']
            required_spending = 0.0
            cash_flow = contribution # Positive cash flow
            
        # Post-Retirement Phase (Decumulation)
        elif age >= info.retirement_age:
            
            # Calculate the nominal required spending for this year
            if age == info.retirement_age:
                 required_spending = nominal_spending_at_retirement
            else:
                 # Spending grows by the scenario's inflation rate each subsequent year
                 required_spending = current_annual_spending * (1 + inflation_rate)
            
            # Update the tracked annual spending for the next iteration
            current_annual_spending = required_spending
            
            cash_flow = -required_spending # Negative cash flow (withdrawal)
        
        # --- 2. UPDATE BALANCE ---
        
        # If the balance is already 0, it remains 0 (cannot go more negative than 0 in this model)
        if current_balance <= 0:
            final_balance = 0
        else:
            # Step A: Apply cash flow (contribution or withdrawal)
            balance_after_cash_flow = current_balance + cash_flow
            
            # Step B: Apply investment growth for the year
            # Note: We still apply growth even if the balance is negative, but the final_balance is capped at 0 for plotting
            current_balance = balance_after_cash_flow * (1 + inv_growth_rate)
            final_balance = max(0, current_balance)

        yearly_data.append({
            'Age': age,
            f'Balance ({scenario_name})': final_balance
        })

    # Merge the results back to the combined DataFrame
    df_temp = pd.DataFrame(yearly_data)
    df_combined = pd.merge(df_combined, df_temp[['Age', f'Balance ({scenario_name})']], on='Age', how='left')

df_combined = df_combined.sort_values(by='Age')

# --- D. Generate Plotly Chart ---
fig = create_projection_chart(df_combined, info.retirement_age, info.death_age)

# To display the interactive graph, you typically save it as an HTML file.
# You can view the output in a web browser.
output_file = "financial_projection_chart.html"
fig.write_html(output_file, full_html=True)

print(f"\n--- Plotly Chart Generated ---")
print(f"Interactive chart saved to: {output_file}")
print("Open this file in a browser to view the graph.")


--- Plotly Chart Generated ---
Interactive chart saved to: financial_projection_chart.html
Open this file in a browser to view the graph.
